## Preparando o ambiente

In [ ]:
# Bibliotecas para conectar com a API do YouTube e manipulação de dados
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON
from dateutil import parser

# Bibliotecas para visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.backends.backend_pdf import PdfPages
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk

# Segurança para chave da API do YouTube
import os
from dotenv import load_dotenv

# Baixa as stopwords da biblioteca nltk, caso ainda não tenha feito isso
nltk.download('stopwords')

# Define o conjunto de stopwords em português
stop_words = set(stopwords.words('portuguese'))

# Carrega variáveis de ambiente, incluindo a chave da API
load_dotenv() 
api_key = os.getenv("YOUTUBE_API_KEY")

# Definições para API do YouTube
channel_ids = ['UC9cz05xObaFpB8U72t73IFA']
api_service_name = "youtube"
api_version = "v3"

# Inicializa o cliente da API do YouTube
youtube = build(api_service_name, api_version, developerKey=api_key)

## Link com Youtube API 

In [ ]:
# Função para obter estatísticas de um canal
def get_channel_stats(youtube, channel_ids):
    all_data = []
    
    # Solicitação à API para informações sobre o canal
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    # Extrai as informações necessárias de cada canal
    for item in response['items']:
        data = {
            'channelName': item['snippet']['title'],
            'subscribers': item['statistics']['subscriberCount'],
            'views': item['statistics']['viewCount'],
            'totalVideos': item['statistics']['videoCount'],
            'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
        
    return pd.DataFrame(all_data)

# Coleta estatísticas do canal
channel_stats = get_channel_stats(youtube, channel_ids)
channel_stats

# ID da playlist de uploads do canal
playlist_id = 'UU9cz05xObaFpB8U72t73IFA'

# Função para obter IDs de vídeos de uma playlist
def get_video_ids(youtube, playlist_id):
    video_ids = []
    
    # Solicitação inicial para a playlist
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults=50
    )
    response = request.execute()
    
    # Armazena IDs de vídeos e lida com múltiplas páginas
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
    
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()
        
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        
        next_page_token = response.get('nextPageToken')
    
    return video_ids

# Coleta IDs dos vídeos
video_ids = get_video_ids(youtube, playlist_id)
len(video_ids)

# Função para obter detalhes de vídeos individuais
def get_video_details(youtube, video_ids):
    all_video_info = []
    
    # Processa em lotes de até 50 vídeos devido ao limite da API
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()
        
        # Extrai informações relevantes para cada vídeo
        for video in response['items']:
            stats_to_keep = {
                'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                'contentDetails': ['duration', 'definition', 'caption']
            }
            
            video_info = {'video_id': video['id']}
            for k, v_list in stats_to_keep.items():
                for v in v_list:
                    try:
                        video_info[v] = video[k][v]
                    except KeyError:
                        video_info[v] = None
            
            all_video_info.append(video_info)
                         
    return pd.DataFrame(all_video_info)

# Coleta detalhes dos vídeos
video_df = get_video_details(youtube, video_ids)

# Pré-processamento dos dados
video_df.isnull().any()
video_df.dtypes

# Converte colunas numéricas para tipo numérico
numeric_cols = ['viewCount', 'likeCount', 'favouriteCount', 'commentCount']
video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Adiciona dia da semana em que o vídeo foi publicado
video_df['publishedAt'] = video_df['publishedAt'].apply(lambda x: parser.parse(x))
video_df['publishDayName'] = video_df['publishedAt'].apply(lambda x: x.strftime("%A"))

# Converte a duração do vídeo para segundos
import isodate
video_df['durationSecs'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x))
video_df['durationSecs'] = video_df['durationSecs'].astype('timedelta64[s]')

# Conta o número de tags por vídeo
video_df['tagCount'] = video_df['tags'].apply(lambda x: 0 if x is None else len(x))

## Criando os gráficos

In [ ]:
# Análise e visualização dos dados

# Gráfico de barras para visualizações dos top 9 vídeos
ax = sns.barplot(x='title', y='viewCount', data=video_df.sort_values('viewCount', ascending=False)[0:9])
ax.tick_params(axis='x', rotation=90)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}K'.format(x / 1000)))

# Distribuição das visualizações por canal
sns.violinplot(x='channelTitle', y='viewCount', data=video_df)

# Gráficos de dispersão para análise de correlação
fig, ax = plt.subplots(1, 2)
sns.scatterplot(data=video_df, x='commentCount', y='viewCount', ax=ax[0])
sns.scatterplot(data=video_df, x='likeCount', y='viewCount', ax=ax[1])

# Limpeza do título dos vídeos para geração de nuvem de palavras
stop_words = set(stopwords.words('portuguese'))
video_df['title_no_stopwords'] = video_df['title'].apply(lambda x: [item for item in str(x).split() if item not in stop_words])
all_words_str = ' '.join([a for b in video_df['title_no_stopwords'].tolist() for a in b])

# Função para plotar a nuvem de palavras
def plot_cloud(wordcloud):
    plt.figure(figsize=(30, 20))
    plt.imshow(wordcloud)
    plt.axis("off")

# Gera a nuvem de palavras com os títulos dos vídeos
wordcloud = WordCloud(width=2000, height=1000, random_state=1, background_color='black',
                      colormap='viridis', collocations=False).generate(all_words_str)
plot_cloud(wordcloud)

# Conta o número de publicações por dia da semana
day_df = pd.DataFrame(video_df['publishDayName'].value_counts()).rename(columns={'publishDayName': 'count'})
day_df.index.name = 'day'

# Reordena para seguir a sequência dos dias da semana
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_df = day_df.reindex(weekdays)

# Gráfico de barras para contagem de publicações por dia da semana
ax = day_df.reset_index().plot.bar(x='day', y='count', rot=0)

# Gera relatório em PDF com visualizações
def generate_pdf_report(video_df):
    with PdfPages("youtube_report_final_design.pdf") as pdf:
        plt.rcParams.update({
            'figure.facecolor': 'white',
            'axes.facecolor': 'white',
            'savefig.facecolor': 'white',
            'axes.edgecolor': 'gray',
            'grid.color': 'lightgray',
            'axes.grid': True,
            'axes.titleweight': 'bold',
            'axes.titlesize': 14,
            'axes.labelsize': 12,
        })

        fig_size = (15, 5)

        # Gráfico 1: Visualizações dos top 9 vídeos
        fig, ax = plt.subplots(figsize=(20, 5))
        sns.barplot(x='title', y='viewCount', data=video_df.sort_values('viewCount', ascending=False)[0:9], ax=ax)
        ax.tick_params(axis='x', rotation=90)
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}K'.format(x / 1000)))
        ax.set_title("Top 9 Vídeos por Contagem de Visualizações")
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()

        # Gráfico 2: Distribuição de visualizações por canal
        fig, ax = plt.subplots(figsize=(20, 5))
        sns.violinplot(x='channelTitle', y='viewCount', data=video_df, ax=ax, inner="quartile")
        ax.set_title("Distribuição de Visualizações por Canal")
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()

        # Gráfico 3: WordCloud dos títulos
        fig, ax = plt.subplots(figsize=(31, 10))
        wordcloud = WordCloud(width=800, height=400, random_state=1, background_color='white', colormap='viridis').generate(all_words_str)
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis("off")
        ax.set_title("WordCloud dos Títulos")
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()

        # Gráfico 4: Publicações por dia da semana
        fig, ax = plt.subplots(figsize=(20, 10))
        day_df.reset_index().plot.bar(x='day', y='count', rot=0, ax=ax)
        ax.set_title("Contagem de Publicações por Dia da Semana")
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()

# print("Relatório PDF gerado com sucesso como 'youtube_report_final_design.pdf'.")

## Gerando o relatorio em PDF

In [ ]:
generate_pdf_report(video_df)